In [2]:
from google.colab import drive
drive.mount('/content/gdrive' )

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/COMP4900/a3/'
!ls './data/'

/content/gdrive/My Drive/COMP4900/a3
Test.pkl  TrainLabels.csv  Train.pkl


In [4]:

# basic
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from keras.utils import plot_model, to_categorical
from keras.callbacks import History
from keras.optimizers import Nadam, Adam, SGD
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder





In [5]:
train_labels = pd.read_csv('./data/TrainLabels.csv')
train_images = pd.read_pickle('./data/Train.pkl')
test_images = pd.read_pickle('./data/Test.pkl')

# train_images = np.where(train_images<250, 0, train_images)
# test_images = np.where(test_images<250, 0, test_images)

X_train = np.array(train_images)
X_train = np.array(X_train).reshape(-1,64,128,1)
onehot_encoder = OneHotEncoder(sparse=False)
y_train = onehot_encoder.fit_transform(np.reshape(np.array(train_labels['class']), (-1,1)))

X_test = np.array(test_images)
X_test = np.array(X_test).reshape(-1,64,128,1)


print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)
print("y_train.shape:",y_train.shape)




X_train.shape: (60000, 64, 128, 1)
X_test.shape: (10000, 64, 128, 1)
y_train.shape: (60000, 9)


#base model 8 layer (maxPolling layers aren't counted here)


In [6]:
baseModel = Sequential([
                        ZeroPadding2D((1,1),input_shape=(64,128,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),

                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.5),

                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.5),

                        Flatten(),
                        Dense(512, activation='relu'),
                        Dropout(0.5),
                        Dense(9, activation='softmax'),

])

baseModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

baseModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 66, 130, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 128, 32)       320       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 66, 130, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 128, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 64, 32)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 34, 66, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 64, 64)        1

In [6]:
history = baseModel.fit(X_train, y_train, batch_size=100, epochs=20, validation_split = 0.33)

print('Finished Training')

predictions = np.argmax(baseModel.predict(X_test), axis=1)

Epoch 1/20
402/402 [==============================] - 10s 26ms/step - loss: 2.2938 - accuracy: 0.1140 - val_loss: 2.1906 - val_accuracy: 0.1408
Epoch 2/20
402/402 [==============================] - 10s 25ms/step - loss: 2.0906 - accuracy: 0.1771 - val_loss: 1.8296 - val_accuracy: 0.2706
Epoch 3/20
402/402 [==============================] - 10s 25ms/step - loss: 1.7752 - accuracy: 0.2746 - val_loss: 1.5987 - val_accuracy: 0.3288
Epoch 4/20
402/402 [==============================] - 10s 25ms/step - loss: 1.5694 - accuracy: 0.3377 - val_loss: 1.4627 - val_accuracy: 0.3742
Epoch 5/20
402/402 [==============================] - 10s 25ms/step - loss: 1.4611 - accuracy: 0.3788 - val_loss: 1.3744 - val_accuracy: 0.4390
Epoch 6/20
402/402 [==============================] - 10s 25ms/step - loss: 1.3815 - accuracy: 0.4178 - val_loss: 1.2438 - val_accuracy: 0.4979
Epoch 7/20
402/402 [==============================] - 10s 25ms/step - loss: 1.3174 - accuracy: 0.4472 - val_loss: 1.2039 - val_accuracy:

# 10 Layers & dropreate =0.5 

In [7]:
model2 = Sequential([
                        ZeroPadding2D((1,1),input_shape=(64,128,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),

                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.5),

                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.5),
                     
                        ZeroPadding2D((1,1)),
                        Conv2D(256, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(256, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.5),

                        Flatten(),
                        Dense(512, activation='relu'),
                        Dropout(0.5),
                        Dense(9, activation='softmax'),

])

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_6 (ZeroPaddin (None, 66, 130, 1)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 128, 32)       320       
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 66, 130, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 128, 32)       9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 64, 32)        0         
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin (None, 34, 66, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 64, 64)       

In [8]:
history = model2.fit(X_train, y_train, batch_size=100, epochs=20, validation_split = 0.33)

print('Finished Training')

predictions2 = np.argmax(model2.predict(X_test), axis=1)

Epoch 1/20
402/402 [==============================] - 21s 52ms/step - loss: 2.2614 - accuracy: 0.1102 - val_loss: 2.1982 - val_accuracy: 0.1107
Epoch 2/20
402/402 [==============================] - 20s 50ms/step - loss: 2.1975 - accuracy: 0.1115 - val_loss: 2.1974 - val_accuracy: 0.1108
Epoch 3/20
402/402 [==============================] - 20s 50ms/step - loss: 2.1975 - accuracy: 0.1105 - val_loss: 2.1974 - val_accuracy: 0.1108
Epoch 4/20
402/402 [==============================] - 20s 50ms/step - loss: 2.1974 - accuracy: 0.1114 - val_loss: 2.1974 - val_accuracy: 0.1103
Epoch 5/20
402/402 [==============================] - 20s 50ms/step - loss: 2.1977 - accuracy: 0.1098 - val_loss: 2.1973 - val_accuracy: 0.1103
Epoch 6/20
402/402 [==============================] - 20s 50ms/step - loss: 2.1972 - accuracy: 0.1152 - val_loss: 2.1846 - val_accuracy: 0.1449
Epoch 7/20
402/402 [==============================] - 20s 50ms/step - loss: 1.9058 - accuracy: 0.2304 - val_loss: 1.5947 - val_accuracy:

# 10 Layers & dropreate =0.3 plus earlystopping

In [ ]:
model3 = Sequential([
                        ZeroPadding2D((1,1),input_shape=(64,128,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(32, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),

                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(64, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.3),

                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(128, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.3),
                     
                        ZeroPadding2D((1,1)),
                        Conv2D(256, kernel_size=(3, 3), activation='relu'),
                        ZeroPadding2D((1,1)),
                        Conv2D(256, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Dropout(0.3),

                        Flatten(),
                        Dense(512, activation='relu'),
                        Dropout(0.3),
                        Dense(9, activation='softmax'),

])

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 66, 130, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 128, 32)       320       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 66, 130, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 128, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 64, 32)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 34, 66, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 64, 64)        1

In [ ]:


from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=3, mode="auto",)
history = model3.fit(X_train, y_train, batch_size=100, epochs=20, validation_split = 0.33, callbacks=[callback])


print('Finished Training')

predictions3 = np.argmax(model3.predict(X_test), axis=1)

Epoch 1/20
402/402 [==============================] - 12s 30ms/step - loss: 2.2378 - accuracy: 0.1086 - val_loss: 2.1973 - val_accuracy: 0.1098
Epoch 2/20
402/402 [==============================] - 11s 28ms/step - loss: 2.1974 - accuracy: 0.1102 - val_loss: 2.1973 - val_accuracy: 0.1103
Epoch 3/20
402/402 [==============================] - 11s 28ms/step - loss: 2.1976 - accuracy: 0.1060 - val_loss: 2.1974 - val_accuracy: 0.1096
Epoch 4/20
402/402 [==============================] - 11s 28ms/step - loss: 2.1974 - accuracy: 0.1115 - val_loss: 2.1936 - val_accuracy: 0.1270
Epoch 5/20
402/402 [==============================] - 11s 28ms/step - loss: 1.9575 - accuracy: 0.2110 - val_loss: 1.6084 - val_accuracy: 0.3523
Epoch 6/20
402/402 [==============================] - 11s 28ms/step - loss: 1.5455 - accuracy: 0.3537 - val_loss: 1.4326 - val_accuracy: 0.4346
Epoch 7/20
402/402 [==============================] - 11s 28ms/step - loss: 1.3949 - accuracy: 0.4284 - val_loss: 1.2754 - val_accuracy:

# Simple model

In [9]:
# model.add(Conv2D(10, kernel_size=(3, 3), activation='relu'))     
#     model.add(MaxPooling2D(pool_size=(2, 2)))                       
#     model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dense(9, activation='softmax'))


simpleModel = Sequential([
                        ZeroPadding2D((1,1),input_shape=(64,128,1)),
                        Conv2D(10, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),
                        Conv2D(20, kernel_size=(3, 3), activation='relu'),
                        MaxPooling2D(pool_size=(2, 2)),


                        Flatten(),
                        Dense(512, activation='relu'),
                        Dense(9, activation='softmax'),

])

simpleModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

simpleModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 66, 130, 1)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 128, 10)       100       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 64, 10)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 30, 62, 20)        1820      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 31, 20)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9300)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)              

In [10]:
history_Simple = simpleModel.fit(X_train, y_train, batch_size=100, epochs=20, validation_split = 0.33)

print('Finished Training')

predictions_Simple = np.argmax(simpleModel.predict(X_test), axis=1)

Epoch 1/20
402/402 [==============================] - 4s 9ms/step - loss: 3.2128 - accuracy: 0.1123 - val_loss: 2.1999 - val_accuracy: 0.1137
Epoch 2/20
402/402 [==============================] - 4s 9ms/step - loss: 2.1638 - accuracy: 0.1482 - val_loss: 2.1341 - val_accuracy: 0.1592
Epoch 3/20
402/402 [==============================] - 4s 9ms/step - loss: 1.9770 - accuracy: 0.2282 - val_loss: 1.9798 - val_accuracy: 0.2193
Epoch 4/20
402/402 [==============================] - 3s 9ms/step - loss: 1.7288 - accuracy: 0.3254 - val_loss: 1.9539 - val_accuracy: 0.2321
Epoch 5/20
402/402 [==============================] - 4s 9ms/step - loss: 1.4465 - accuracy: 0.4442 - val_loss: 2.0829 - val_accuracy: 0.2304
Epoch 6/20
402/402 [==============================] - 3s 9ms/step - loss: 1.0618 - accuracy: 0.6045 - val_loss: 2.3744 - val_accuracy: 0.2155
Epoch 7/20
402/402 [==============================] - 3s 9ms/step - loss: 0.6807 - accuracy: 0.7546 - val_loss: 2.9181 - val_accuracy: 0.2063
Epoch 

In [ ]:
results = np.array(list(zip(np.arange(0,10000),predictions3+5)))
results = pd.DataFrame(results, columns=['id', 'class'])
results.to_csv('submit34.csv', index = False)